##Langkah langkah




# Data Preparation


1. **Normalisasi Data dan Pembersihan**: Metode normalisasi dan pembersihan data sangat penting untuk meningkatkan kualitas klasifikasi, terutama dalam mengelola fitur yang tidak seragam pada dataset gambar. Penelitian menunjukkan bahwa normalisasi dan analisis komponen utama (PCA) dapat meningkatkan pemisahan kelas dan akurasi klasifikasi secara keseluruhan [(Polyakova & Krylov, 2022)](https://consensus.app/papers/data-normalization-methods-improve-quality-polyakova/d05c23cb4aa057359d8ea5247d98ecbb/?utm_source=chatgpt).

2. **Augmentasi Data**: Augmentasi, seperti rotasi, flipping, dan zooming, telah terbukti efektif untuk memperluas variasi data. Teknik ini meningkatkan akurasi model dengan mencegah overfitting saat jumlah data terbatas. Metode augmentasi berbasis pasangan gambar dapat memperbaiki hasil klasifikasi dalam dataset besar dan kecil [(Inoue, 2018)](https://consensus.app/papers/data-augmentation-pairing-samples-images-classification-inoue/2b6f58f0f2ec5a3f8678dd3fccf679e0/?utm_source=chatgpt) dan [(Meethongjan et al., 2022)](https://consensus.app/papers/data-augmentation-combining-feature-selection-color-meethongjan/2483aed2d63a54719e70fc9ef3b295af/?utm_source=chatgpt).

3. **Pembagian Dataset**: Memisahkan dataset menjadi bagian training, validation, dan testing membantu mengevaluasi model secara akurat. Penelitian terkait menyarankan proporsi yang seimbang untuk memaksimalkan generalisasi model, khususnya dalam dataset yang besar seperti ImageNet [(Mensink et al., 2013)](https://consensus.app/papers/distancebased-image-classification-generalizing-mensink/04f5f7362310529fa561411c004c86ae/?utm_source=chatgpt).

4. **Pengelolaan Dataset Besar**:
Mengelola data gambar dalam dataset besar, seperti CIFAR-10 atau ImageNet, sering kali memerlukan teknik kompresi atau pengaturan untuk mengatasi batasan memori dan kecepatan pemrosesan. Optimalisasi algoritma pengurutan dan penerapan hashing terbukti dapat mempercepat klasifikasi tanpa mengorbankan akurasi (Cao et al., 2023).

In [ ]:
!pip install tensorflow numpy matplotlib datasets wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datasets
import wandb
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers, models


In [ ]:
# Muat dataset menggunakan Hugging Face
dataset = datasets.load_dataset("garythung/trashnet")

# Melihat beberapa data untuk mendapatkan gambaran
dataset['train'][0]


dataset-original.zip:  24%|##3       | 860M/3.63G [00:00<?, ?B/s]

dataset-resized.zip:   0%|          | 0.00/42.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5054 [00:00<?, ? examples/s]

{'image': <PIL.Image.Image image mode=RGB size=3024x4032>, 'label': 0}

In [ ]:
# Menyiapkan fungsi normalisasi
def preprocess_image(image):
    image = tf.image.resize(image, [224, 224])  # Sesuaikan ukuran gambar
    image = image / 255.0  # Normalisasi pixel value ke rentang 0-1
    return image

# Mengaplikasikan fungsi preprocess pada dataset
train_images = [preprocess_image(image['image']) for image in dataset['train']]


In [ ]:
# Augmentasi data menggunakan ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=30,  # Rotasi gambar
    width_shift_range=0.2,  # Geser lebar
    height_shift_range=0.2,  # Geser tinggi
    shear_range=0.2,
    zoom_range=0.2,  # Zoom gambar
    horizontal_flip=True,  # Membalik gambar secara horizontal
    fill_mode='nearest'  # Mengisi ruang kosong setelah transformasi
)

# Contoh augmentasi pada gambar pertama
augmented_image = datagen.random_transform(train_images[0])
plt.imshow(augmented_image)
plt.show()


In [ ]:
# Membangun model CNN dengan TensorFlow
model = models.Sequential([
    layers.InputLayer(input_shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')  # Sesuaikan dengan jumlah kelas pada dataset
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Pembagian data menjadi train, validation, dan test
X_train, X_val, y_train, y_val = train_test_split(train_images, dataset['train']['label'], test_size=0.2, random_state=42)

# Untuk pengujian, kita akan menggunakan 10% dari data sebagai test set
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [ ]:
# Evaluasi model pada data test
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test))
print(f"Test accuracy: {test_acc}")


In [ ]:
# Melakukan PCA untuk analisis komponen utama (PCA)
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(np.array(X_train).reshape(len(X_train), -1))

# Visualisasi komponen utama
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis')
plt.colorbar()
plt.show()


In [ ]:
wandb.init(project="trash_classification")

# Melatih model dengan wandb
history = model.fit(
    np.array(X_train), np.array(y_train),
    epochs=10,
    validation_data=(np.array(X_val), np.array(y_val)),
    callbacks=[wandb.keras.WandbCallback()]
)

# Log final metrics
wandb.log({"test_accuracy": test_acc})
wandb.finish()


In [ ]:
# Menyimpan model
model.save('trash_classification_model')

# Mengupload model ke Hugging Face Hub
from transformers import AutoModelForImageClassification, AutoTokenizer
# Kode ini membutuhkan setup Hugging Face API untuk upload model


In [ ]:
tensorflow
numpy
matplotlib
datasets
wandb


#Exploratory Image Analysis:
1. **Visualisasi Gambar: Visualisasi adalah langkah penting untuk memahami
distribusi dan variasi dalam dataset gambar**. Menurut beberapa studi, melihat contoh-contoh dari setiap kelas membantu memahami perbedaan visual yang mungkin mempengaruhi model klasifikasi (Zhou et al., 2019).
2. **Statistik Dasar: Menghitung jumlah gambar per kelas dan ukuran gambar adalah langkah krusial untuk memahami keseimbangan data dan potensi bias.** Penelitian menunjukkan bahwa distribusi data yang tidak seimbang dapat menyebabkan model lebih cenderung untuk mendeteksi kelas dengan representasi data yang lebih banyak (Zhu & Yang, 2021).


 ## Pelatihan dan Evaluasi Model CNN
 **Pelatihan dan Evaluasi Model CNN** untuk klasifikasi gambar, berikut adalah beberapa referensi terkait setiap tahap yang dapat membantu dalam memahami pendekatan terbaik:

1. **Membangun Model CNN**: Convolutional Neural Networks (CNNs) terbukti sangat efektif dalam klasifikasi gambar dengan beberapa arsitektur populer seperti AlexNet, VGG, dan ResNet yang menawarkan performa tinggi melalui beberapa lapisan konvolusi dan pooling. Sebagai contoh, penelitian menunjukkan bagaimana CNN dapat menangkap fitur multi-skala dari gambar yang relevan untuk klasifikasi dengan hasil yang baik pada dataset kompleks [(Han et al., 2018)](https://consensus.app/papers/image-classification-method-using-transfer-learning-data-han/963bcdde94af525c8e01a98cc684fe6f/?utm_source=chatgpt).

2. **Pelatihan Model**: Untuk optimasi pelatihan model CNN, algoritma seperti Adam dan Stochastic Gradient Descent (SGD) sering digunakan. Penelitian membandingkan efektivitas metode optimasi ini pada berbagai ukuran filter konvolusi (misalnya 3x3, 7x7) dan menyarankan bahwa Adam sering memiliki konvergensi yang lebih cepat, terutama pada dataset dengan variasi data yang tinggi [(Jaworska, 2018)](https://consensus.app/papers/image-segment-classification-using-jaworska/d9573d4d0c6c57a1a6fc796724e3e0fa/?utm_source=chatgpt).

3. **Evaluasi Model**: Metrik seperti akurasi, precision, recall, dan F1-score adalah esensial untuk mengevaluasi performa model klasifikasi gambar. Dalam penelitian pada klasifikasi gambar multi-kelas, pendekatan ini menunjukkan keefektifan dalam menilai kemampuan model dalam mengklasifikasikan dengan benar setiap kelas, bahkan pada dataset besar seperti CIFAR-10 [(Yang et al., 2021)](https://consensus.app/papers/twostage-selective-ensemble-deep-tree-training-medical-yang/c6cc0dd3c58f587ba3d73f4ae03a268c/?utm_source=chatgpt).

Referensi-referensi ini memberikan landasan kuat untuk memahami dan mengimplementasikan CNN dalam klasifikasi gambar, dari pembangunan model hingga evaluasi yang akurat terhadap performa model.

# Beberapa Insight perlu diperhatikan

Berikut adalah beberapa referensi yang relevan mengenai **Bias dalam Model** dan **Masalah dalam Dataset** untuk klasifikasi gambar, khususnya dalam memeriksa dan mengurangi bias yang disebabkan oleh distribusi data yang tidak seimbang atau label yang salah:

1. **Bias dalam Model**: Ketidakseimbangan dalam dataset dapat menyebabkan model belajar bias yang tidak diinginkan. Pendekatan seperti teknik augmentasi berbasis tekstur dan transfer gaya digunakan untuk memperbaiki bias dan meningkatkan generalisasi model terhadap data di luar distribusi pelatihan. Ini memungkinkan model untuk mengurangi ketergantungan pada fitur bias dari data pelatihan [(Kang et al., 2021)](https://consensus.app/papers/data-generation-using-texture-cooccurrence-spatial-kang/5e30056d02f252a394e40d4b51169104/?utm_source=chatgpt).

2. **Evaluasi Bias melalui Eksplanasi Model**: Menggunakan metode penjelasan model, seperti analisis visual, dapat membantu mendeteksi bias dalam klasifikasi gambar. Analisis ini memungkinkan identifikasi fitur spesifik yang dipelajari model sebagai akibat dari bias pada data pelatihan, sehingga membantu mengurangi ketergantungan pada atribut sensitif [(Tong & Kagal, 2020)](https://consensus.app/papers/investigating-bias-image-classification-using-model-tong/f9231d853d5c56228850751f215b76db/?utm_source=chatgpt).

3. **Masalah dalam Dataset dan Ketidakcocokan Label**: Seringkali, bias muncul karena label yang tidak sesuai atau distribusi data yang tidak mewakili kelas tertentu dengan baik. Metode pembagian dataset berbasis atribut dapat membantu dalam memitigasi bias dengan menghindari pengaruh fitur yang tidak relevan, sehingga meningkatkan akurasi dalam kondisi dunia nyata [(Wolf et al., 2023)](https://consensus.app/papers/addressing-bias-finegrained-classification-datasets-wolf/dc26e9ba145f53288fe1521837219394/?utm_source=chatgpt).

4. **Penyesuaian untuk Bias dalam Dataset**: Model berbasis diskriminatif dapat belajar untuk mengatasi bias dataset melalui teknik yang memisahkan "world weights" dari.